In [ ]:
from LSTM_model import LSTM
import numpy as np
import random
import torch
import time
from datetime import datetime, timedelta
from collections import deque
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS


In [ ]:
# Setting random seed for reproducibility
torch.manual_seed(140)
np.random.seed(140)
random.seed(140)

In [ ]:
# Functions for RePAD2
def calculate_aare(actual, predicted):
    """
    Calculate the Average Absolute Relative Error (AARE) between actual and predicted values.

    Parameters:
    - actual (list or array): The actual values.
    - predicted (list or array): The predicted values.

    Returns:
    - float: The AARE value.
    """
    if len(actual) != len(predicted):
        raise ValueError("The length of actual and predicted values must be the same.")

    # Calculate the absolute relative errors
    absolute_relative_errors = [abs((a - p) / a) for a, p in zip(actual, predicted) if a != 0]

    # Calculate the average of these errors
    aare = sum(absolute_relative_errors) / len(absolute_relative_errors)
    
    return aare

def calculate_threshold(aare_values):
    """
    Calculate the threshold value (Thd) based on an array of AARE values.
    Thd is defined as the mean of the AARE values plus three times their standard deviation.

    Parameters:
    - aare_values (array-like): An array of AARE values.

    Returns:
    - float: The calculated threshold value (Thd).
    """
    # Calculate the mean and standard deviation of the AARE values
    mean_aare = np.mean(aare_values)
    std_aare = np.std(aare_values)
    
    # Calculate Thd
    thd = mean_aare + 3 * std_aare
    
    return thd

# Function for creating model
def train_model(train_events):
    tensor_y = torch.tensor(train_events, dtype=torch.float32).view(-1, 1, 1)
    tensor_x = torch.tensor([1, 2, 3], dtype=torch.float32).view(-1, 1, 1)
    # Create an instance of the LSTM model
    model = LSTM(tensor_x, tensor_y, input_size=1, hidden_size=10, num_layers=1, output_size=1, batch_size=3, num_epochs=50, learning_rate=0.005)
    model.train_model() # Train the model

    return model

def report_anomaly(value, timestamp, write_api):
    """
    Sends an anomalous event back to InfluxDB, storing it in the "anomaly" measurement
    with both the same value and time as the original event.

    Parameters:
    - anomalous_event: The event data that was detected as an anomaly, including its value and timestamp.
    """

    point = Point("anomaly")\
        .tag("host", "MIURA")\
        .field("value", value)\
        .time(timestamp, WritePrecision.NS)
    
    write_api.write(bucket="system_state", org="ORG", record=point)
    print(f"Anomalous event sent to InfluxDB: Value={value}, Time={timestamp}")

In [ ]:
### REPAD2 Algorithm ###

"""
THE PLAN

Making the program start from any predefined time, and then continue to fetch data from that time and onwards.
And so it can process older data and then catch up to the present time.

It therefore processes a batch of 3 from the earliest timestamp in the range, and then updates the start time for the next iteration 
by incrementing the start time by a set time to both avoid duplicate events and to eventually catch up to the present time.

The program will run indefinitely, and will continue to fetch data from the InfluxDB and process it in batches of 3 until the program is stopped.
I there are not enough events for a batch, the program will wait for a set time before trying again.
When a batch of three is available it will be processed and the it will again wait for another event to be available.

This way it is both flexible and efficient, and can be easily used to process either data in real-time or historical data


For each batch it follows the algorithm of RePAD2


To-Do:

Store actual values (Store whole event?) with predicted values in sliding window 

"""

# Testing of InfluxDB with LSTM
# Må installeres influxdb-client "pip install influxdb-client"
# https://www.influxdata.com/blog/getting-started-with-python-and-influxdb-v2-0/
influxdb_url = "http://localhost:8086"
token = "random_token"
username = "influx-admin"
password = "ThisIsNotThePasswordYouAreLookingFor"
org = "ORG"
bucket = "system_state"
measurement = "cpu_utilization"

# Instantiate the QueryAPI
client = InfluxDBClient(url=influxdb_url, token=token, org=org, username=username, password=password)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

# Sliding window for Threshold
#sliding_window = deque(maxlen=8064)
actual_values = deque(maxlen=8064)
predicted_values = deque(maxlen=8064)
sliding_window_AARE = deque(maxlen=8064)

# Time parameters
poll_interval = 1
time_increment = 1
start_time = "2014-04-10T00:00:00Z"

# RePAD2 specific
T = 0
flag = True 
M = None # Model


while True:

    # Construct the Flux query
    query = f'''
    from(bucket: "{bucket}")
     |> range(start: time(v: "{start_time}"))
     |> filter(fn: (r) => r["_measurement"] == "{measurement}")
    '''
        
    # Query the data
    events = list(query_api.query_stream(org=org, query=query))

    if len(events) >= 4: # Need at least 4 to predict next and compare
        for i in range(len(events) - 3):
            batch_events = events[i:i+3]
            next_event = events[i+3]

            # Testing    
            T += 1
            print(f'T: {T}')
            #time.sleep(1)

        # Set T to the length of the sliding 
            if T >= 2 and T < 5: # 3 total values when T = 2
                M = train_model([event.get_value() for event in list(batch_events)])
                pred_D_T_plus_1 = M.predict_next()
                # Append the event and its prediction to the sliding window
                actual_values.append(next_event.get_value())
                predicted_values.append(pred_D_T_plus_1)

            elif T >= 5 and T < 7:
                # Calculate AARE and append to sliding window
                AARE_T = calculate_aare(actual_values, predicted_values)
                sliding_window_AARE.append(AARE_T)
                M = train_model([event.get_value() for event in list(batch_events)])
                pred_D_T_plus_1 = M.predict_next()
                # Append the event and its prediction to the sliding window
                actual_values.append(next_event.get_value())
                predicted_values.append(pred_D_T_plus_1)

                #print(f'Pred_D_T: {pred_D_T_plus_1}')

            elif T >= 7 and flag == True:
                if T != 7: # Use M to precdict D_T
                    pred_D_T = M.predict_next()
                    # Append the event and its prediction to the sliding window
                    actual_values.append(next_event.get_value())
                    predicted_values.append(pred_D_T)

                # Calculate AARE and append to sliding window
                AARE_T = calculate_aare(actual_values, predicted_values)
                sliding_window_AARE.append(AARE_T)
                # Calculate Thd
                Thd = calculate_threshold(sliding_window_AARE)
                
                if AARE_T <= Thd: pass # Calculate AARE and append to sliding window
                else:
                    # Pop the right of the sliding window
                    actual_values.pop()
                    predicted_values.pop()
                    # Train an LSTM model with D_T-3, D_T-2, D_T-1
                    model = train_model([event.get_value() for event in list(batch_events)])
                    # Use the model to predict D_T
                    pred_D_T = model.predict_next()
                    
                    # Append the event and its prediction to the sliding window
                    actual_values.append(next_event.get_value())
                    predicted_values.append(pred_D_T)

                    # Re-calculate AARE_T
                    AARE_T = calculate_aare(actual_values, predicted_values)
                    sliding_window_AARE.append(AARE_T)
                    # Re-calculate Thd
                    Thd = calculate_threshold(sliding_window_AARE)

                    if AARE_T <= Thd:
                        # D_T is not reported as anomaly
                        # Replace M with the new model
                        M = model
                        # Update flag to True
                        flag = True

                    else:
                        # D_T reported as anomaly immediately
                        report_anomaly(next_event.get_value(), next_event.get_time(), write_api)
                        # Update flag to False
                        flag = False
            
            elif T >= 7 and flag == False:
                # Train an LSTM model with D_T-3, D_T-2, D_T-1
                model = train_model([event.get_value() for event in list(batch_events)])
                # Use the model to predict D_T
                pred_D_T = model.predict_next()
                # Append the event and its prediction to the sliding window
                actual_values.append(next_event.get_value())
                predicted_values.append(pred_D_T)

                # Calculate AARE_T
                AARE_T = calculate_aare(actual_values, predicted_values)
                sliding_window_AARE.append(AARE_T)
                # Calculate Thd
                Thd = calculate_threshold(sliding_window_AARE)

                if AARE_T <= Thd:
                    # D_T is not reported as anomaly
                    # Replace M with the new model
                    M = model
                    # Update flag to True
                    flag = True

                else:
                    # D_T reported as anomaly immediately
                    report_anomaly(next_event.get_value(), next_event.get_time(), write_api)
                    # Update flag to False
                    flag = False


            if T > 2:
                # Print newest value in the predicted_values deque
                print(f'Predicted value: {predicted_values[-1]}')


        # Update start time for the next iteration
        last_event_time = batch_events[-1].get_time()
        # Increment by 1 second to avoid duplicate events
        start_time = (last_event_time + timedelta(seconds=time_increment)).isoformat()
                

    else:
        print("No events found in range.")

    time.sleep(poll_interval)
